Double-check the filters proposed. 

In [1]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [335]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

Get echos that can be associated to an icustay_id. 

In [55]:
query = \
"""
SELECT *
FROM echo_icustay
"""
echos = pd.read_sql_query(query, con)
echos.shape

(25512, 7)

In [72]:
echos.head()

,row_id,charttime,icustay_id,hadm_id,subject_id,intime,outtime
0,67775,2170-11-08 12:03:00,206613,106296,269,2170-11-05 11:05:29,2170-11-08 17:46:57
1,73508,2186-12-25 23:54:00,210407,164716,272,2186-12-25 21:08:04,2186-12-27 12:01:13
2,99418,2114-07-03 11:54:00,254851,130546,274,2114-06-28 22:28:44,2114-07-07 18:01:16
3,75764,2164-06-15 10:26:00,204407,192224,279,2164-06-14 21:18:27,2164-06-18 22:53:12
4,60894,2101-10-24 09:50:00,257572,111199,281,2101-10-18 04:45:22,2101-10-25 22:29:25


In [73]:
len(echos['icustay_id'].unique())

19752

In [74]:
len(echos['hadm_id'].unique())

19244

In [75]:
len(echos['subject_id'].unique())

17122

Compute variables required for filtering, namely
+ Time of echo with respect to icustay intime
+ Whether or not patient was on a vasopressor during the icustay
+ Whether or not patient has an excluded diagnosis during the hospital admission
+ Age on admission to the ICU

Check the two hardest. Firstly, on whether or not the patient was on a vasopressor during the icustay: Look up prescriptions, filter for prescriptions that are vasopressors, count the number of unique icustay_ids. 

In [80]:
query = \
"""
SELECT ps.* FROM prescriptions ps
INNER JOIN d_prescriptions_vaso dpv
ON (ps.drug IS NOT DISTINCT FROM dpv.drug
    AND ps.drug_name_poe IS NOT DISTINCT FROM dpv.drug_name_poe
    AND ps.drug_name_generic IS NOT DISTINCT FROM dpv.drug_name_generic
    AND ps.route IS NOT DISTINCT FROM dpv.route)
INNER JOIN echo_icustay ei
ON ps.icustay_id = ei.icustay_id
;
"""
ps = pd.read_sql_query(query, con)
ps.head()

,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route
0,2350267,25,129635,203487,2160-11-02,2160-11-03,MAIN,Norepinephrine,None,None,LEVO4I,004939,00074144304,4MG/4ML AMP,4,mg,1,AMP,IV DRIP
1,1379347,21,111970,216859,2135-01-31,2135-02-06,MAIN,Norepinephrine,None,None,LEVO4I,028633,00703115303,4mg/4mL Amp,8,mg,2,AMP,IV DRIP
2,1379366,21,111970,216859,2135-02-05,2135-02-06,MAIN,Vasopressin,None,None,VASO20I,006611,61570042001,20U/ML VIAL,100,UNIT,5,ml,IV DRIP
3,1381496,36,182104,280987,2131-05-04,2131-05-04,MAIN,Phenylephrine,None,None,PHEN12SYR,005066,00409180001,12MG/50ML SYR,12,mg,1,SYR,IV DRIP
4,2907879,35,166707,282039,2122-02-10,2122-02-11,MAIN,Phenylephrine,None,None,PHEN12SYR,005066,00409180001,12MG/50ML SYR,12,mg,1,SYR,IV DRIP


In [87]:
len(ps['icustay_id'].unique())

8425

In [267]:
query = \
"""
SELECT * FROM echo_filter_vars
"""
efv = pd.read_sql_query(query, con)
efv.head()

,row_id,charttime,icustay_id,hadm_id,subject_id,intime,outtime,intime_to_echo,echo_to_outtime,ps_vaso,diag_xc,age_at_intime,dob
0,83837,2182-08-19 10:46:00,264827,195409,32755,2182-08-18 16:50:19,2182-08-19 16:33:46,0 days 17:55:41,0 days 05:47:46,False,True,21643 days 16:50:19,2123-04-30
1,104824,2119-01-26 14:49:00,276795,124493,69851,2119-01-26 16:57:32,2119-01-27 19:18:02,-1 days +21:51:28,1 days 04:29:02,False,True,25047 days 16:57:32,2050-06-09
2,59659,2200-10-22 15:14:00,217810,188268,82208,2200-10-21 14:54:57,2200-10-24 14:01:58,1 days 00:19:03,1 days 22:47:58,False,True,22331 days 14:54:57,2139-08-15
3,67634,2186-09-19 12:09:00,263932,168160,24182,2186-09-19 09:40:07,2186-09-22 13:15:57,0 days 02:28:53,3 days 01:06:57,True,True,27636 days 09:40:07,2110-12-29
4,97569,2183-06-24 12:57:00,296461,107856,15977,2183-06-24 15:44:23,2183-06-28 16:12:29,-1 days +21:12:37,4 days 03:15:29,False,True,20237 days 15:44:23,2128-01-12


Ok. The numbers agree now. This is the number of icustays associated with ps_vaso. 

In [95]:
len(efv.loc[efv['ps_vaso'], 'icustay_id'].unique())

8425

In [97]:
len(efv.loc[efv['ps_vaso'], 'hadm_id'].unique())

8318

In [96]:
len(efv.loc[efv['ps_vaso'], 'subject_id'].unique())

7897

Also, check whether or not the patient has an excluded diagnosis. To do that, get the list of diagnoses to be excluded, filter the list of diagnoses actually to be excluded. 

In [98]:
query = \
"""
SELECT * FROM d_diagnoses_xc_annot
"""
dxc = pd.read_sql_query(query, con)
dxc.head()

,icd9_code,short_title,long_title,k_card,k_heart,k_hemorrhag,k_bleed,k_embolism,k_shock,k_clot,num,exclude,comments
0,3641,Meningococc pericarditis,Meningococcal pericarditis,1,0,0,0,0,0,0,11,1,None
1,4082,Toxic shock syndrome,Toxic shock syndrome,0,0,0,0,0,1,0,25,0,None
2,521,Varicella pneumonitis,Varicella (hemorrhagic) pneumonitis,0,0,1,0,0,0,0,4,1,None
3,860,Chagas disease of heart,Chagas' disease with heart involvement,0,1,0,0,0,0,0,1,1,None
4,11281,Candidal endocarditis,Candidal endocarditis,1,0,0,0,0,0,0,6,1,None


In [99]:
dxc_icd9s = set(dxc.loc[dxc['exclude'] == 1, 'icd9_code'].unique())

In [100]:
query = \
"""
SELECT * FROM DIAGNOSES_ICD
"""
dg = pd.read_sql_query(query, con)
dg.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [101]:
dg_ = dg.loc[dg['icd9_code'].apply(lambda d: d in dxc_icd9s)]
dg_.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code
4,1301,109,172335,5.0,4254
28,1488,112,174105,1.0,53100
29,1489,112,174105,2.0,41071
51,1511,115,114585,10.0,4280
67,1527,117,140784,5.0,4280


In [104]:
len(set(dg_['hadm_id'].unique()) & set(echos['hadm_id']))

14125

In [106]:
len(efv.loc[efv['diag_xc'], 'icustay_id'].unique())

14586

In [105]:
len(efv.loc[efv['diag_xc'], 'hadm_id'].unique())

14125

In [107]:
len(efv.loc[efv['diag_xc'], 'subject_id'].unique())

12587

Ok. I think it agrees. Summarize in spreadsheet with the various filters on. 

In [111]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'], 'icustay_id'].unique())

2328

In [112]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'], 'hadm_id'].unique())

2315

In [113]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'], 'subject_id'].unique())

2277

Time filter

In [179]:
time_filter = (efv['intime_to_echo'] > pd.Timedelta('-8 hours')) & (efv['intime_to_echo'] < pd.Timedelta('48 hours'))
len(efv.loc[time_filter, 'icustay_id'].unique())

16467

In [182]:
len(efv.loc[time_filter, 'hadm_id'].unique())

16115

In [183]:
len(efv.loc[time_filter, 'subject_id'].unique())

14567

On pressors, no excluded diagnoses, and time filter

In [184]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'] & time_filter, 'icustay_id'].unique())

1967

In [187]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'] & time_filter, 'hadm_id'].unique())

1964

In [188]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'] & time_filter, 'subject_id'].unique())

1944

Age

In [251]:
efv.shape

(25512, 12)

In [258]:
ef.shape

(25512, 14)

In [270]:
ef.head()

,row_id,charttime,icustay_id,hadm_id,subject_id,intime,outtime,intime_to_echo,echo_to_outtime,ps_vaso,diag_xc,age_at_intime,dob,time_filter,age_filter
0,83837,2182-08-19 10:46:00,264827,195409,32755,2182-08-18 16:50:19,2182-08-19 16:33:46,0 days 17:55:41,0 days 05:47:46,False,True,21643 days 16:50:19,2123-04-30,True,True
1,104824,2119-01-26 14:49:00,276795,124493,69851,2119-01-26 16:57:32,2119-01-27 19:18:02,-1 days +21:51:28,1 days 04:29:02,False,True,25047 days 16:57:32,2050-06-09,True,True
2,59659,2200-10-22 15:14:00,217810,188268,82208,2200-10-21 14:54:57,2200-10-24 14:01:58,1 days 00:19:03,1 days 22:47:58,False,True,22331 days 14:54:57,2139-08-15,True,True
3,67634,2186-09-19 12:09:00,263932,168160,24182,2186-09-19 09:40:07,2186-09-22 13:15:57,0 days 02:28:53,3 days 01:06:57,True,True,27636 days 09:40:07,2110-12-29,True,True
4,97569,2183-06-24 12:57:00,296461,107856,15977,2183-06-24 15:44:23,2183-06-28 16:12:29,-1 days +21:12:37,4 days 03:15:29,False,True,20237 days 15:44:23,2128-01-12,True,True


In [287]:
ef.loc[(ef['age_at_intime'] > np.timedelta64('18', 'Y')) != ef['age_filter']].head()

,row_id,charttime,icustay_id,hadm_id,subject_id,intime,outtime,intime_to_echo,echo_to_outtime,ps_vaso,diag_xc,age_at_intime,dob,time_filter,age_filter
8,62763,2149-02-07 10:30:00,243283,175360,20249,2149-02-06 15:54:53,2149-02-10 07:33:42,0 days 18:35:07,2 days 21:03:42,False,True,-103152 days +16:20:19.290448,1846-10-04,True,True
113,87360,2102-04-07 10:59:00,228956,111845,57889,2102-04-06 17:07:33,2102-04-14 15:47:50,0 days 17:51:27,7 days 04:48:50,False,True,-104004 days +17:32:59.290448,1802-04-06,True,True
132,88989,2105-07-28 15:42:00,295698,155806,62637,2105-07-22 20:58:19,2105-07-29 22:58:25,5 days 18:43:41,1 days 07:16:25,False,True,-104004 days +21:23:45.290448,1805-07-22,False,True
165,67394,2142-03-12 15:39:00,274086,152254,8533,2142-03-11 17:10:51,2142-03-16 14:45:25,0 days 22:28:09,3 days 23:06:25,False,True,-102155 days +17:36:17.290448,1837-02-15,True,True
177,61844,2120-01-17 10:53:00,229322,149689,24933,2120-01-16 14:42:46,2120-01-24 17:49:51,0 days 20:10:14,7 days 06:56:51,False,True,-104002 days +15:08:12.290448,1820-01-14,True,True


In [285]:
query = \
"""
SELECT * FROM icustays ic
INNER JOIN patients pt
ON ic.subject_id = pt.subject_id
WHERE ic.subject_id = '20249'; 
"""
check = pd.read_sql_query(query, con)
check.head()

,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,25703,20249,152504,206946,carevue,CCU,CCU,57,57,2146-10-04 16:05:22,2146-10-06 18:52:58,2.1164,19108,20249,M,1846-10-04,2149-02-09,2149-02-09,2149-02-09,1
1,25704,20249,175360,243283,carevue,CCU,CCU,7,7,2149-02-06 15:54:53,2149-02-10 07:33:42,3.6520,19108,20249,M,1846-10-04,2149-02-09,2149-02-09,2149-02-09,1


Ok. Age is really awkward because of the older patients whose age are computed as negative numbers, for some reason. It looks like sql deals with is correctly, but somehow my python code does not. 

In [323]:
age_filter = (ef['age_at_intime'] > np.timedelta64('18', 'Y')) | (ef['age_at_intime'] < np.timedelta64(0))

On pressors + no excluded diagnosis + time + age

In [324]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'] & time_filter & age_filter, 'icustay_id'].unique())

1954

In [325]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'] & time_filter & age_filter, 'hadm_id'].unique())

1951

In [326]:
len(efv.loc[efv['ps_vaso'] & ~efv['diag_xc'] & time_filter & age_filter, 'subject_id'].unique())

1931

Break down the diagnosis exclusions. 

In [131]:
query = \
"""
SELECT ei.icustay_id, ei.hadm_id, ei.subject_id
    ,dx.icd9_code, dx.long_title
FROM d_diagnoses_xc_annot dx
INNER JOIN diagnoses_icd di
    ON di.icd9_code = dx.icd9_code
INNER JOIN echo_icustay ei
    ON di.hadm_id = ei.hadm_id -- join on hospital admission
WHERE dx.exclude = 1
"""
dxc = pd.read_sql_query(query, con)
dxc.head()

,icustay_id,hadm_id,subject_id,icd9_code,long_title
0,262652,172335,109,4254,Other primary cardiomyopathies
1,232514,114585,115,4280,"Congestive heart failure, unspecified"
2,227264,195632,123,41011,Acute myocardial infarction of other anterior ...
3,225073,134369,124,9971,"Cardiac complications, not elsewhere classified"
4,256064,138376,124,42833,Acute on chronic diastolic heart failure


In [166]:
dxc_ = pd.concat([dxc[['icustay_id', 'icd9_code', 'long_title']].drop_duplicates().groupby(['icd9_code', 'long_title']).count(), 
           dxc[['hadm_id', 'icd9_code', 'long_title']].drop_duplicates().groupby(['icd9_code', 'long_title']).count(), 
           dxc[['subject_id', 'icd9_code', 'long_title']].drop_duplicates().groupby(['icd9_code', 'long_title']).count()], 
           axis = 1)
dxc_.to_csv("../../data/generated/D_DIAGNOSES_XC_COUNTS.csv")
dxc_.head()

,,icustay_id,hadm_id,subject_id
icd9_code,long_title,,,
11281,Candidal endocarditis,5,4,3
1510,Malignant neoplasm of cardia,14,14,14
1641,Malignant neoplasm of heart,1,1,1
2127,Benign neoplasm of heart,31,30,30
2851,Acute posthemorrhagic anemia,1482,1421,1401


In [165]:
dxc_.sort_values('icustay_id', ascending = False).head(n = 10)

,,icustay_id,hadm_id,subject_id
icd9_code,long_title,,,
4280,"Congestive heart failure, unspecified",7269,7021,6135
41071,"Subendocardial infarction, initial episode of care",2221,2123,2048
2851,Acute posthemorrhagic anemia,1482,1421,1401
9971,"Cardiac complications, not elsewhere classified",1472,1422,1409
42789,Other specified cardiac dysrhythmias,1135,1111,1089
4271,Paroxysmal ventricular tachycardia,1112,1069,1043
78551,Cardiogenic shock,1025,972,951
4275,Cardiac arrest,959,895,888
4254,Other primary cardiomyopathies,952,911,838


Perhaps count the number of excluded diagnosis assigned to each id. 

In [160]:
dxc[['icustay_id', 'icd9_code']].drop_duplicates().groupby('icustay_id').count()['icd9_code'].value_counts().sort_index()

1     5053
2     4365
3     2795
4     1409
5      599
6      215
7       91
8       43
9       14
10       1
11       1
Name: icd9_code, dtype: int64

In [162]:
dxc[['hadm_id', 'icd9_code']].drop_duplicates().groupby('hadm_id').count()['icd9_code'].value_counts().sort_index()

1     4972
2     4235
3     2691
4     1335
5      567
6      196
7       81
8       37
9        9
10       1
11       1
Name: icd9_code, dtype: int64

In [163]:
dxc[['subject_id', 'icd9_code']].drop_duplicates().groupby('subject_id').count()['icd9_code'].value_counts().sort_index()

1     4344
2     3532
3     2276
4     1244
5      630
6      286
7      139
8       74
9       38
10       8
11       8
12       5
13       3
Name: icd9_code, dtype: int64

Quick count

In [327]:
query = \
"""
SELECT * FROM echo_filtered; 
"""
ef = pd.read_sql_query(query, con)
ef.head()

,row_id,charttime,icustay_id,hadm_id,subject_id,intime,outtime,intime_to_echo,echo_to_outtime,ps_vaso,diag_xc,age_at_intime,dob,time_filter,age_filter
0,62570,2102-04-14 10:00:00,207317,191210,44605,2102-04-14 08:58:39,2102-04-15 17:17:53,01:01:21,1 days 07:17:53,True,False,23809 days 08:58:39,2037-01-21,True,True
1,68949,2162-06-11 08:53:00,230215,153438,3778,2162-06-11 10:38:17,2162-06-13 18:31:49,-1 days +22:14:43,2 days 09:38:49,True,False,25791 days 10:38:17,2091-10-10,True,True
2,84167,2191-03-15 00:22:00,279743,180621,19896,2191-03-14 22:55:19,2191-04-05 13:05:09,01:26:41,21 days 12:43:09,True,False,31859 days 22:55:19,2103-11-30,True,True
3,64922,2181-07-02 11:11:00,290910,170691,59945,2181-07-02 10:41:19,2181-07-03 23:01:06,00:29:41,1 days 11:50:06,True,False,24202 days 10:41:19,2115-03-11,True,True
4,80126,2173-02-09 09:13:00,221265,112693,26548,2173-02-09 10:42:28,2173-02-10 14:25:27,-1 days +22:30:32,1 days 05:12:27,True,False,21091 days 10:42:28,2115-04-23,True,True


In [329]:
len(ef['icustay_id'].unique())

1954

In [330]:
len(ef['hadm_id'].unique())

1951

In [331]:
len(ef['subject_id'].unique())

1931